In [11]:
import os
import numpy as np
import pandas as pd
import tempfile

import pandapower as pp
import pandapower.networks as nw
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import pandapower.plotting as pplt
import matplotlib.pyplot as plt
from pandapower.plotting.plotly import pf_res_plotly

#creating our net
net = nw.create_cigre_network_mv()

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')

#print(net)

#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, color ="g", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()

#Adding CS at rated power (6.6 kW)


max_line_loading=100

max_vm_pu= 1.1

min_vm_pu= 0.9

In [12]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,Trafo 0-1,None,0,1,25.00,110.0,20.0,12.00107,0.160,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
1,Trafo 0-12,None,0,12,25.00,110.0,20.0,12.00107,0.160,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True
2,CS 14 - 0.63MVA,0.63 MVA 20/0.4 kV,14,15,0.63,20.0,0.4,6.00000,1.206,1.65,...,0.0,-2.0,2.0,2.5,0.0,0.0,False,1,1.0,True
3,CS 11 - 0.4MVA,0.4 MVA 20/0.4 kV,11,16,0.40,20.0,0.4,6.00000,1.425,1.35,...,0.0,-2.0,2.0,2.5,0.0,0.0,False,1,1.0,True
4,CS 5 - 0.4MVA,0.4 MVA 20/0.4 kV,5,17,0.40,20.0,0.4,6.00000,1.425,1.35,...,0.0,-2.0,2.0,2.5,0.0,0.0,False,1,1.0,True
5,CS 4 - 0.25MVA,0.25 MVA 20/0.4 kV,4,18,0.25,20.0,0.4,6.00000,1.440,0.80,...,0.0,-2.0,2.0,2.5,0.0,0.0,False,1,1.0,True
6,CS 9 - 0.25MVA,0.25 MVA 20/0.4 kV,9,19,0.25,20.0,0.4,6.00000,1.440,0.80,...,0.0,-2.0,2.0,2.5,0.0,0.0,False,1,1.0,True


In [13]:
#importing the active and reactive power of loads from excel
df_active = pd.read_csv('active_power_time_series_ev_load.csv')
df_reactive = pd.read_csv('reactive_power_time_series_ev_load.csv')
# create the data source from
ds_active = DFData(df_active)
ds_reactive = DFData(df_reactive)

def timeseries_example(output_dir):
    
    #defing the time step
    n_timesteps = 24
    
    # 3. create controllers (to control P values of the load) ds_p is for active power and ds_q for reactive power
    create_controllers(net, ds_active, ds_reactive)

    # time steps to be calculated. Could also be a list with non-consecutive time steps
    time_steps = range(0, n_timesteps)

    # 4. the output writer with the desired results to be stored to files.
    ow = create_output_writer(net, time_steps, output_dir=output_dir)

    # 5. the main time series function
    run_timeseries(net, time_steps) 
    
    
load_index=net.load.index
load_name=net.load.name
def create_controllers(net,ds_active,ds_reactive):
    for i in range(len(load_index)):
        ConstControl(net, element='load', variable='p_mw', element_index=load_index[i],
                 data_source=ds_active, profile_name=load_name[i])
        ConstControl(net, element='load', variable='q_mvar', element_index=load_index[i],
                 data_source=ds_reactive, profile_name=load_name[i])

def create_output_writer(net, time_steps, output_dir):
    ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())
    # these variables are saved to the harddisk after / during the time series loop
    ow.log_variable('res_load', 'p_mw')
    ow.log_variable('res_bus', 'vm_pu')
    ow.log_variable('res_line', 'loading_percent')
    ow.log_variable('res_line', 'i_ka')
    ow.log_variable('res_trafo','loading_percent')
    return ow


In [14]:
for i in range (1,93):
    l=str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
#Bus CS 11 (0.4 MVA) - trafo id: 3
for i in range (93,150):
    l=str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

#Bus CS 5 (0.4 MVA) - trafo id: 4
for i in range (150,207):
    l=str(i)
    pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (207,242):
    l=str(i)
    pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

In [ ]:
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:01<00:00, 32.39it/s]

In [ ]:
ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)
    
vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

if any(line_loading.max()>100):
        print("Overloaded lines")

elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")

In [ ]:
print("-----------------------------    STEP 1.2    -----------------------")

#net = nw.create_cigre_network_mv()

vmax = 1.1
vmin = 0.9
max_ll = 100

lines = net.line.index 
N = list()

for l in lines:
    net.line.loc[l,"in_service"] = False
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)
    
    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
   # pp.runpp(net)
    
    if any(vm_pu.max() > vmax) or any(vm_pu.min() < vmin) or any(line_loading) > max_ll:
        N.append(l)
        print("critical line")
        print(l)
       
    net.line.loc[l,"in_service"] = True
    
#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, N, color ="r", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()
print(N)

In [37]:
N

[0, 1, 10, 11]

In [54]:
print("-----------------------------    STEP 1.2    -----------------------")

#net = nw.create_cigre_network_mv()

vmax = 1.1
vmin = 0.9
max_ll = 100

lines = net.line.index 
N_1 = list()

net.switch.closed[1] = True
net.switch.closed[2] = True
net.switch.closed[4] = True

for l in lines:
    net.line.loc[l,"in_service"] = False
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)
    
    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
   # pp.runpp(net)
    
    if any(vm_pu.max() > vmax) or any(vm_pu.min() < vmin) or any(line_loading) > max_ll:
        N_1.append(l)
        print("critical line")
        print(l)
       
    net.line.loc[l,"in_service"] = True
    
#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, N, color ="r", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()
print(N_1)

-----------------------------    STEP 1.2    -----------------------
Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:04<00:00,  5.32it/s]


critical line
0
Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:04<00:00,  5.14it/s]


critical line
1
Results can be found in your local temp folder: time_series_ev


...

Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:05<00:00,  4.62it/s]


Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:05<00:00,  4.09it/s]

[0, 1, 10, 11]


In [ ]:
print("-----------------------------    STEP 1.2    -----------------------")

#net = nw.create_cigre_network_mv()

vmax = 1.1
vmin = 0.9
max_ll = 100

lines = net.line.index 
N_2 = list()
net.switch.closed[1] = False
net.switch.closed[2] = False
net.switch.closed[4] = False

for l in lines:
    net.line.loc[l,"in_service"] = False
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)
    
    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
   # pp.runpp(net)
    
    if any(vm_pu.max() > vmax) or any(vm_pu.min() < vmin) or any(line_loading) > max_ll:
        N_2.append(l)
        print("critical line")
        print(l)
       
    net.line.loc[l,"in_service"] = True
    
#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, N, color ="r", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()
print(N_2)

In [55]:
print("-----------------------------    STEP 1.2    -----------------------")

#net = nw.create_cigre_network_mv()

vmax = 1.1
vmin = 0.9
max_ll = 100

lines = net.line.index 
N_3 = list()
net.switch.closed[1] = False
net.switch.closed[2] = True
net.switch.closed[4] = True

for l in lines:
    net.line.loc[l,"in_service"] = False
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)
    
    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
   # pp.runpp(net)
    
    if any(vm_pu.max() > vmax) or any(vm_pu.min() < vmin) or any(line_loading) > max_ll:
        N_3.append(l)
        print("critical line")
        print(l)
       
    net.line.loc[l,"in_service"] = True
    
#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, N, color ="r", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()
print(N_3)

-----------------------------    STEP 1.2    -----------------------
Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:08<00:00,  2.81it/s]


critical line
0
Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:08<00:00,  2.93it/s]


critical line
1
Results can be found in your local temp folder: time_series_ev


...

Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:08<00:00,  2.94it/s]


Results can be found in your local temp folder: time_series_ev


100%|██████████| 24/24 [00:10<00:00,  2.35it/s]

[0, 1, 10, 11]


In [ ]:
print("-----------------------------    STEP 1.2    -----------------------")

#net = nw.create_cigre_network_mv()

vmax = 1.1
vmin = 0.9
max_ll = 100

lines = net.line.index 
N_4 = list()
net.switch.closed[1] = True
net.switch.closed[2] = False
net.switch.closed[4] = True

for l in lines:
    net.line.loc[l,"in_service"] = False
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)
    
    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
   # pp.runpp(net)
    
    if any(vm_pu.max() > vmax) or any(vm_pu.min() < vmin) or any(line_loading) > max_ll:
        N_4.append(l)
        print("critical line")
        print(l)
       
    net.line.loc[l,"in_service"] = True
    
#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, N, color ="r", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()
print(N_4)

In [ ]:
print("-----------------------------    STEP 1.2    -----------------------")

#net = nw.create_cigre_network_mv()

vmax = 1.1
vmin = 0.9
max_ll = 100

lines = net.line.index 
N_5 = list()
net.switch.closed[1] = True
net.switch.closed[2] = True
net.switch.closed[4] = False

for l in lines:
    net.line.loc[l,"in_service"] = False
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)
    
    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
   # pp.runpp(net)
    
    if any(vm_pu.max() > vmax) or any(vm_pu.min() < vmin) or any(line_loading) > max_ll:
        N_5.append(l)
        print("critical line")
        print(l)
       
    net.line.loc[l,"in_service"] = True
    
#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, N, color ="r", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()
print(N_5)

In [ ]:
print("-----------------------------    STEP 1.2    -----------------------")

#net = nw.create_cigre_network_mv()

vmax = 1.1
vmin = 0.9
max_ll = 100

lines = net.line.index 
N_6 = list()
net.switch.closed[1] = False
net.switch.closed[2] = False
net.switch.closed[4] = True

for l in lines:
    net.line.loc[l,"in_service"] = False
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)
    
    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
   # pp.runpp(net)
    
    if any(vm_pu.max() > vmax) or any(vm_pu.min() < vmin) or any(line_loading) > max_ll:
        N_6.append(l)
        print("critical line")
        print(l)
       
    net.line.loc[l,"in_service"] = True
    
#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, N, color ="r", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()
print(N_6)